In [2]:
import cv2
import numpy as np
import scipy
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import glob
from tqdm import tqdm

In [3]:
def real_and_phase(file_path):
    
    df = pd.read_csv(file_path)

    # Identify columns for magnitude and phase by checking if 'Phase' is in the column name
    mag_cols = [col for col in df.columns if 'Phase' not in col and col != 'Unnamed: 0' and col != 'Roundness']
    phase_cols = [col for col in df.columns if 'Phase' in col]

    # Create empty DataFrames for real and imaginary components with the 'name' column
    df_real = pd.DataFrame()
    df_imag = pd.DataFrame()

    # Copy the names of the images into the 'name' column for both DataFrames
    df_real['name'] = df['Unnamed: 0']
    df_imag['name'] = df['Unnamed: 0']

    # Loop through each column pair (magnitude and phase) to calculate real and imaginary components
    for mag_col, phase_col in zip(mag_cols, phase_cols):
        # Calculate real and imaginary components
        real_values = df[mag_col] * np.cos(df[phase_col])
        imag_values = df[mag_col] * np.sin(df[phase_col])
    
        # Store the calculated values in the respective DataFrames
        df_real[mag_col] = real_values
        df_imag[mag_col] = imag_values
        
    return df_real, df_imag

In [4]:
def recon_image(df_real, df_imag, file_path, desired_aspect_ratio=(6, 6)):
    
    os.makedirs(file_path, exist_ok=True)

    # Loop through each row to recreate the image using inverse Fourier Transform
    for index, (real_row, imag_row) in enumerate(zip(df_real.iterrows(), df_imag.iterrows())):
            
        _, real_data = real_row
        _, imag_data = imag_row
        
        # Extract the name of the image from the 'name' column
        image_name = real_data['name']
        
        # Remove the 'name' element to only keep the Fourier Descriptor values
        real_values = real_data.drop(labels=['name']).values
        imag_values = imag_data.drop(labels=['name']).values
        
        # Construct Fourier Descriptor coefficients from real and imaginary parts
        FD_coeff = real_values + 1j * imag_values
        
        # Perform inverse Fourier Transform
        spatial_coords = np.fft.ifft(np.fft.ifftshift(FD_coeff * len(FD_coeff)))
        
        # Extract X and Y coordinates
        x_coords = spatial_coords.real
        y_coords = spatial_coords.imag
        
        # Calculate the aspect ratio of the original image
        original_aspect_ratio = (max(x_coords) - min(x_coords)) / (max(y_coords) - min(y_coords))
        
        # Set the figure size to match the desired aspect ratio
        fig_width, fig_height = desired_aspect_ratio
        
        # Adjust the figure size to maintain the original aspect ratio
        if original_aspect_ratio > 1:
            fig_height /= original_aspect_ratio
        else:
            fig_width *= original_aspect_ratio
        
        # Create the plot with the fixed size and calculated aspect ratio
        fig, ax = plt.subplots(figsize=(fig_width, fig_height))
        ax.fill(x_coords, y_coords, 'k-', alpha=1.0)  # Black color
        ax.axis('off')  # Turn off axis
        ax.set_facecolor('white')  # Set background color to white
        
        # Save the figure in the 'file_path' folder with a name based on the image name
        save_path = os.path.join(file_path, 'Recon_' + f'{image_name}')
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0, facecolor='white', dpi=350)
        plt.close()

In [ ]:
file_path = 'SRC-J FD.csv'

df = pd.read_csv(file_path)

mag_cols = [col for col in df.columns if 'Phase' not in col and col != 'Unnamed: 0' and col != 'Roundness']
phase_cols = [col for col in df.columns if 'Phase' in col]

for i in tqdm(range(21,80)):
    df_real = pd.DataFrame()
    df_imag = pd.DataFrame()
    df_real['name'] = df['Unnamed: 0']
    df_imag['name'] = df['Unnamed: 0']
    mag = mag_cols[ 80 - i : 81 + i ]
    phase = phase_cols[ 80 - i : 81 + i ]
    
    for mag, phase in zip(mag, phase):
        # Calculate real and imaginary components
        real_values = df[mag] * np.cos(df[phase])
        imag_values = df[mag] * np.sin(df[phase])
    
        # Store the calculated values in the respective DataFrames
        df_real[mag] = real_values
        df_imag[mag] = imag_values
        
    name = str(-i+1)+' to '+str(i+1)
    recon_image(df_real,df_imag,name)